In [2]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 6.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 10.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=b04ce80cf439fde31d21a0f6ca76c15f636f33a22e34d6a05bd8153dcf1bae09
  Stored in directory: /home/codespace/.cache/pip/wheels/9c/aa/b1/8433fd8b1afe7eb31196cc74a42cd778bcb52636a428da079d
Successfully built pyspark


In [1]:
import findspark
from pyspark.sql import SparkSession
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import re

In [2]:
spark_session = SparkSession.builder.appName('data_film').getOrCreate()

22/12/08 19:43:10 WARN Utils: Your hostname, codespaces-90c577 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
22/12/08 19:43:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/08 19:43:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark_session.read.option("delimiter", "\t").csv('data_film.txt', header=False)
data = data.toDF('titre', 'date', 'duree', 'type', 'note', 'nombre avis', 'avis')

# Preprocess

In [4]:
data.printSchema()

root
 |-- titre: string (nullable = true)
 |-- date: string (nullable = true)
 |-- duree: string (nullable = true)
 |-- type: string (nullable = true)
 |-- note: string (nullable = true)
 |-- nombre avis: string (nullable = true)
 |-- avis: string (nullable = true)



In [5]:
from scripts import preprocess_for_ML

data = preprocess_for_ML.preproces_for_machine_learning(data, spark_session)